In [13]:
from pycaret.datasets import get_data
data = get_data(dataset='traffic')

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,Rush Hour,traffic_volume
0,None,288.28,0.0,0.0,40,Clouds,1,5545
1,None,289.36,0.0,0.0,75,Clouds,0,4516
2,None,289.58,0.0,0.0,90,Clouds,0,4767
3,None,290.13,0.0,0.0,90,Clouds,0,5026
4,None,291.14,0.0,0.0,75,Clouds,0,4918


In [14]:
data_train = data.sample(frac=0.9, random_state=123)
datanew    = data.drop(data_train.index)

In [15]:
from pycaret.regression import *

In [16]:
s = setup(data_train, target='traffic_volume', session_id=1)

,Description,Value
0,Session id,1
1,Target,traffic_volume
2,Target type,Regression
3,Original data shape,"(43384, 8)"
4,Transformed data shape,"(43384, 29)"
5,Transformed train set shape,"(30368, 29)"
6,Transformed test set shape,"(13016, 29)"
7,Numeric features,5
8,Categorical features,2
9,Preprocess,True


In [17]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,1503.6562,3086088.9252,1756.6677,0.2162,0.9268,2.4059,0.3160
gbr,Gradient Boosting Regressor,1533.7772,3159919.1492,1777.5434,0.1975,0.9390,2.4436,1.2250
ada,AdaBoost Regressor,1579.2988,3289143.2760,1813.5322,0.1647,0.9685,2.5984,0.5210
knn,K Neighbors Regressor,1565.0124,3631874.3559,1905.6658,0.0774,0.9518,2.5217,0.5530
rf,Random Forest Regressor,1548.7036,3745267.5897,1935.1141,0.0486,0.9512,2.3858,4.4220
omp,Orthogonal Matching Pursuit,1710.5631,3846297.7494,1961.1558,0.0231,1.0185,2.8528,0.1910
dummy,Dummy Regressor,1742.2076,3938216.2179,1984.4396,-0.0002,1.0281,2.7941,0.1540
et,Extra Trees Regressor,1679.4890,4576530.2223,2139.1418,-0.1626,1.0420,2.4766,3.1230
dt,Decision Tree Regressor,1734.0064,5196451.4188,2279.3680,-0.3200,1.1123,2.3062,0.2480
en,Elastic Net,1678.3732,12410355.8372,2668.1752,-2.1274,1.0005,2.7822,0.1720


In [18]:
tuned = tune_model(estimator=best)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1515.0313,3164513.5283,1778.9080,0.2088,0.9276,1.6915
1,1522.9426,3175190.5867,1781.9064,0.2095,0.9493,2.8195
2,1503.5942,3134374.8257,1770.4166,0.1951,0.9292,2.9351
3,1493.3128,3093367.2926,1758.7971,0.2073,0.9202,4.1479
4,1484.2338,3045933.9152,1745.2604,0.2233,0.9241,2.5114
5,1520.1681,3207303.3281,1790.8946,0.1898,0.9165,1.4436
6,1507.5990,3126638.4538,1768.2303,0.2137,0.9395,3.0210
7,1476.0466,3039898.0423,1743.5303,0.2026,0.9194,2.1290
8,1480.6444,3055185.0431,1747.9088,0.2190,0.9068,1.7446


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [19]:
final_model = finalize_model(estimator=tuned)

In [20]:
final_model

Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['temp', 'rain_1h', 'snow_1h',
                                             'clouds_all', 'Rush Hour'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['holiday', 'weather_main'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('onehot_encoding',
                 TransformerWrapper(include=['holiday', 'weather_main'],
                                    transformer=OneHotEncoder(cols=['holiday',
                                                                    'weather_main'],
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('clean_column_names',
                 TransformerWrapper(transformer=CleanColumnNames())),
                ('actual_estimator', LGBMRegressor(random_state=1))])

In [21]:
save_model(final_model, 'traffic_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['temp', 'rain_1h', 'snow_1h',
                                              'clouds_all', 'Rush Hour'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['holiday', 'weather_main'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('onehot_encoding',
                  TransformerWrapper(include=['holiday', 'weather_main'],
                                     transformer=OneHotEncoder(cols=['holiday',
                                                                     'weather_main'],
                                                               handle_missing='return_nan',
                                                               use_cat_names=True))),
                 ('clean_colum

In [23]:
datanew

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,Rush Hour,traffic_volume
13,None,287.16,0.0,0.0,1,Clear,0,1529
17,None,281.18,0.0,0.0,1,Clear,0,273
22,None,278.12,0.0,0.0,1,Clear,1,6511
37,None,283.17,0.0,0.0,1,Clear,0,312
41,None,280.74,0.0,0.0,1,Clear,0,5689
...,...,...,...,...,...,...,...,...
48134,None,280.08,0.0,0.0,1,Clear,0,2751
48151,None,278.09,0.0,0.0,20,Clouds,0,2713
48180,None,280.07,0.0,0.0,90,Clouds,0,696
48200,None,282.76,0.0,0.0,90,Clouds,0,2781


In [22]:
saved_model = load_model('traffic_model')

Transformation Pipeline and Model Successfully Loaded


In [24]:
predictions = predict_model(saved_model, data=datanew)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,1500.7727,3075281.6162,1753.6481,0.2261,0.8997,1.3860


In [25]:
predictions

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,Rush Hour,traffic_volume,prediction_label
13,None,287.160004,0.0,0.0,1,Clear,0,1529,2469.035546
17,None,281.179993,0.0,0.0,1,Clear,0,273,2364.119169
22,None,278.119995,0.0,0.0,1,Clear,1,6511,4580.540266
37,None,283.170013,0.0,0.0,1,Clear,0,312,2308.466797
41,None,280.739990,0.0,0.0,1,Clear,0,5689,2364.119169
...,...,...,...,...,...,...,...,...,...
48134,None,280.079987,0.0,0.0,1,Clear,0,2751,2364.119169
48151,None,278.089996,0.0,0.0,20,Clouds,0,2713,3163.952418
48180,None,280.070007,0.0,0.0,90,Clouds,0,696,3114.831518
48200,None,282.760010,0.0,0.0,90,Clouds,0,2781,2906.704821
